In [1]:
# classic_crypto_fixed.py
# Implementasi: Caesar, Vigenere, Affine, Playfair, Hill (2x2)

import math
from typing import List, Tuple

ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# -------------------- Utilities --------------------

def sanitize_text(txt: str) -> str:
    return ''.join(ch for ch in txt.upper() if ch.isalpha())

def modinv(a: int, m: int) -> int:
    a = a % m
    if math.gcd(a, m) != 1:
        raise ValueError(f"No modular inverse for {a} mod {m}")
    t0, t1 = 0, 1
    r0, r1 = m, a
    while r1:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        t0, t1 = t1, t0 - q * t1
    return t0 % m

# -------------------- Caesar --------------------

def caesar_encrypt(plaintext: str, shift: int) -> str:
    s = sanitize_text(plaintext)
    out = []
    for ch in s:
        idx = ALPHABET.index(ch)
        out.append(ALPHABET[(idx + shift) % 26])
    return ''.join(out)

def caesar_decrypt(ciphertext: str, shift: int) -> str:
    return caesar_encrypt(ciphertext, -shift)

# -------------------- Vigenere --------------------

def vigenere_encrypt(plaintext: str, key: str) -> str:
    s = sanitize_text(plaintext)
    k = sanitize_text(key)
    out = []
    for i, ch in enumerate(s):
        ki = ALPHABET.index(k[i % len(k)])
        ci = (ALPHABET.index(ch) + ki) % 26
        out.append(ALPHABET[ci])
    return ''.join(out)

def vigenere_decrypt(ciphertext: str, key: str) -> str:
    s = sanitize_text(ciphertext)
    k = sanitize_text(key)
    out = []
    for i, ch in enumerate(s):
        ki = ALPHABET.index(k[i % len(k)])
        pi = (ALPHABET.index(ch) - ki) % 26
        out.append(ALPHABET[pi])
    return ''.join(out)

# -------------------- Affine --------------------

def affine_encrypt(plaintext: str, a: int, b: int) -> str:
    if math.gcd(a, 26) != 1:
        raise ValueError('a must be coprime with 26')
    s = sanitize_text(plaintext)
    out = []
    for ch in s:
        x = ALPHABET.index(ch)
        out.append(ALPHABET[(a * x + b) % 26])
    return ''.join(out)

def affine_decrypt(ciphertext: str, a: int, b: int) -> str:
    a_inv = modinv(a, 26)
    s = sanitize_text(ciphertext)
    out = []
    for ch in s:
        y = ALPHABET.index(ch)
        out.append(ALPHABET[(a_inv * (y - b)) % 26])
    return ''.join(out)

# -------------------- Playfair --------------------

def build_playfair_table(key: str) -> List[List[str]]:
    k = sanitize_text(key).replace('J', 'I')
    seen = []
    for ch in k:
        if ch not in seen:
            seen.append(ch)
    for ch in ALPHABET:
        if ch == 'J':
            continue
        if ch not in seen:
            seen.append(ch)
    table = [seen[i*5:(i+1)*5] for i in range(5)]
    return table

def find_in_table(table: List[List[str]], ch: str) -> Tuple[int, int]:
    for r in range(5):
        for c in range(5):
            if table[r][c] == ch:
                return r, c
    raise ValueError('Char not in table')

def playfair_prepare(plaintext: str) -> List[Tuple[str, str]]:
    s = sanitize_text(plaintext).replace('J', 'I')
    pairs = []
    i = 0
    while i < len(s):
        a = s[i]
        b = s[i+1] if i+1 < len(s) else 'X'
        if a == b:
            pairs.append((a, 'X'))
            i += 1
        else:
            pairs.append((a, b))
            i += 2
    return pairs

def playfair_encrypt(plaintext: str, key: str) -> str:
    table = build_playfair_table(key)
    pairs = playfair_prepare(plaintext)
    out = []
    for a, b in pairs:
        ra, ca = find_in_table(table, a)
        rb, cb = find_in_table(table, b)
        if ra == rb:
            out.append(table[ra][(ca + 1) % 5])
            out.append(table[rb][(cb + 1) % 5])
        elif ca == cb:
            out.append(table[(ra + 1) % 5][ca])
            out.append(table[(rb + 1) % 5][cb])
        else:
            out.append(table[ra][cb])
            out.append(table[rb][ca])
    return ''.join(out)

def playfair_decrypt(ciphertext: str, key: str) -> str:
    table = build_playfair_table(key)
    s = sanitize_text(ciphertext).replace('J', 'I')
    pairs = [(s[i], s[i+1]) for i in range(0, len(s), 2)]
    out = []
    for a, b in pairs:
        ra, ca = find_in_table(table, a)
        rb, cb = find_in_table(table, b)
        if ra == rb:
            out.append(table[ra][(ca - 1) % 5])
            out.append(table[rb][(cb - 1) % 5])
        elif ca == cb:
            out.append(table[(ra - 1) % 5][ca])
            out.append(table[(rb - 1) % 5][cb])
        else:
            out.append(table[ra][cb])
            out.append(table[rb][ca])
    return ''.join(out)

# -------------------- Hill (2x2) --------------------

def hill_encrypt(plaintext: str, key_matrix: List[int]) -> str:
    if len(key_matrix) != 4:
        raise ValueError('Key matrix must have 4 integers (2x2)')
    s = sanitize_text(plaintext)
    if len(s) % 2 == 1:
        s += 'X'
    out = []
    for i in range(0, len(s), 2):
        v = [ALPHABET.index(s[i]), ALPHABET.index(s[i+1])]
        c0 = (key_matrix[0]*v[0] + key_matrix[1]*v[1]) % 26
        c1 = (key_matrix[2]*v[0] + key_matrix[3]*v[1]) % 26
        out.append(ALPHABET[c0])
        out.append(ALPHABET[c1])
    return ''.join(out)

def hill_decrypt(ciphertext: str, key_matrix: List[int]) -> str:
    if len(key_matrix) != 4:
        raise ValueError('Key matrix must have 4 integers (2x2)')
    a, b, c, d = key_matrix
    det = (a*d - b*c) % 26
    det_inv = modinv(det, 26)
    inv_mat = [
        (d * det_inv) % 26,
        (-b * det_inv) % 26,
        (-c * det_inv) % 26,
        (a * det_inv) % 26
    ]
    s = sanitize_text(ciphertext)
    out = []
    for i in range(0, len(s), 2):
        v = [ALPHABET.index(s[i]), ALPHABET.index(s[i+1])]
        p0 = (inv_mat[0]*v[0] + inv_mat[1]*v[1]) % 26
        p1 = (inv_mat[2]*v[0] + inv_mat[3]*v[1]) % 26
        out.append(ALPHABET[p0])
        out.append(ALPHABET[p1])
    return ''.join(out)

# -------------------- Demo --------------------

if __name__ == '__main__':
    print('--- Caesar ---')
    print('Enc:', caesar_encrypt('HELLO WORLD', 3))
    print('Dec:', caesar_decrypt('KHOORZRUOG', 3))

    print('\\n--- Vigenere ---')
    print('Enc:', vigenere_encrypt('ATTACKATDAWN', 'LEMON'))
    print('Dec:', vigenere_decrypt('LXFOPVEFRNHR', 'LEMON'))

    print('\\n--- Affine ---')
    enc = affine_encrypt('HELLO', 5, 8)
    print('Enc:', enc)
    print('Dec:', affine_decrypt(enc, 5, 8))

    print('\\n--- Playfair ---')
    enc = playfair_encrypt('HIDETHEGOLDINTHETREESTUMP', 'PLAYFAIREXAMPLE')
    print('Enc:', enc)
    print('Dec:', playfair_decrypt(enc, 'PLAYFAIREXAMPLE'))

    print('\\n--- Hill ---')
    key = [3, 3, 2, 5]
    enc = hill_encrypt('HELP', key)
    print('Enc:', enc)
    print('Dec:', hill_decrypt(enc, key))


--- Caesar ---
Enc: KHOORZRUOG
Dec: HELLOWORLD
\n--- Vigenere ---
Enc: LXFOPVEFRNHR
Dec: ATTACKATDAWN
\n--- Affine ---
Enc: RCLLA
Dec: HELLO
\n--- Playfair ---
Enc: BMODZBXDNABEKUDMUIXMMOUVIF
Dec: HIDETHEGOLDINTHETREXESTUMP
\n--- Hill ---
Enc: HIAT
Dec: HELP
